In [1]:
%pwd

'/data/longdang/adversarial-train-ParallelComp'

In [2]:
%cd /storage2-mnt/data/longdang/FedShare

/data/longdang/FedShare


In [3]:
%pwd

'/data/longdang/FedShare'

# Requirements

1. python 3.7.6
2. torch 13.1
3. matplotlib
4. h5py
5. tqdm
6. ipykernel

# Import

In [4]:
import torch
from torch import nn 
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, Subset, ConcatDataset
import torch.nn.functional as F
import torch.backends.cudnn as cudnn #ChatGPT: optimize the performance of convolutional neural network computations on NVIDIA GPUs

import copy
import numpy as np
from numpy import linalg as LA
import random
from tqdm import trange
import h5py

from utils.sampling import iid
from utils.options import args_parser

from src.nets import MLP, CNN_v1, CNN_v2, CNN_v3, Alexnet, get_model
from src.strategy import FedAvg
# from src.test import test_img
from src.attack import visualize_adversarial_images
from src.resnet import ResNet18

import pickle
import os
import sys

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# matplotlib.use('Agg')
# plt.style.use('seaborn')
# plt.rcParams.update({'lines.markeredgewidth': 1})

from datetime import datetime

import math
from PIL import Image
import itertools
import collections

/home/l/longdang/.conda/envs/torch_13/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# class Args - Change 3

In [5]:
class Args:
    fed = 'Krum'
    rounds = 2
    num_users = 5 #1 user
    frac = 1
    local_ep = 1 #increase next or K in the paper
    min_le = 5
    max_le = 15
    local_bs = 128
    bs = 128 #test batch size
    lr = 0.001
    global_lr = 1 #SCAFFOLD 
    momentum = 0.9
    classwise = 1000 #maximum sharing 10,000 images.
    alpha = 0
    adv_eps = 0.031
    l2_lambda = 0.0002
    ##########################
    dataset = 'cifar'
    # dataset = 'cifar'
    model='customized_resnet18'
    sampling= 'iid'
    num_classes=10
    num_channels=3
    gpu=0
    verbose=True
    seed=123
    all_clients=True
    sys_homo=True
    debug=True
    device = torch.device('cuda:{}'.format(gpu) if torch.cuda.is_available() and gpu != -1 else 'cpu')
    soft_label_clean = 0.95
    mean = 0
    sigma = 0.0001
    rho = 0.1
    #PGD
    eps=0.0314
    nb_iter=7
    eps_iter=0.00784
    clip_min=0.0
    clip_max=1.0
    #FGSM
    eps_FGSM = 0.031
    adv_global_train_file_name='/data/longdang/FedShare/data/Adv_data/CW/cifar10/adversarial_examples_train_cifar10_3.pkl'
    adv_global_test_file_name='/data/longdang/FedShare/data/Adv_data/CW/cifar10/adversarial_examples_test_cifar10_3.pkl'
    #Krum
    byzantine_threshold=2
args=Args()

# Device

In [6]:
args.device

device(type='cuda', index=0)

# Reproducibility

In [7]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)     
torch.cuda.manual_seed_all(args.seed)

# Load dataset and split users

### train_dg_split

In [8]:
def train_dg_split(dataset, args): 
    dg_idx = []
    train_idx = []
    idxs = np.arange(len(dataset))

    if args.dataset == "mnist":
        labels = dataset.targets.numpy()
    elif args.dataset == "cifar":
        labels = np.array(dataset.targets)
    else:
        exit('Error: unrecognized dataset')
    
    # sort labels
    idxs_labels = np.vstack((idxs, labels))
    idxs_labels = idxs_labels[:,idxs_labels[1,:].argsort()]
    
    #add some adjustment 
    
    idxs = idxs_labels[0]
    labels = idxs_labels[1]
    
    #for all classes
    for i in range(args.num_classes):
        specific_class = np.extract(labels == i, idxs)
        
        # uniformly assign the particular class (args.classwise) to the global dataset
        dg = np.random.choice(specific_class, args.classwise, replace=False)
        
        # divide and split the datata into the global dataset and all parties' dataset
        train_tmp = set(specific_class)-set(dg)
        
        dg_idx = dg_idx + list(dg)
        
        train_idx = train_idx + list(train_tmp)
    
    return dg_idx, train_idx 

In [9]:
def noniid(dataset, args):
    """
    Sample non-I.I.D client data from dataset
    -> Different clients can hold vastly different amounts of data
    :param dataset:
    :param num_users:
    :return:
    """
    num_dataset = len(dataset)
    idx = np.arange(num_dataset)
    dict_users = {i: list() for i in range(args.num_users)}
    
    min_num = int(num_dataset/args.num_users)
    max_num = int(num_dataset/args.num_users)

    random_num_size = np.random.randint(min_num, max_num+1, size=args.num_users)
    print(f"Total number of datasets owned by clients : {sum(random_num_size)}")

    # total dataset should be larger or equal to sum of splitted dataset.
    assert num_dataset >= sum(random_num_size)

    # divide and assign
    for i, rand_num in enumerate(random_num_size):

        rand_set = set(np.random.choice(idx, rand_num, replace=False))
        idx = list(set(idx) - rand_set)
        dict_users[i] = rand_set

    return dict_users

In [10]:
def print_statistics(i, y_train_pi, nb_labels=10):
    print('Party_', i)
    for l in range(nb_labels):
        print('* Label ', l, ' samples: ', (y_train_pi == l).sum())

### CustomTensorDataset

In [11]:
class CustomTensorDataset(Dataset):
    # adv_train_dataset = CustomTensorDataset(train_concatenated_array, 
    #                                         train_reshaped_labels, 
    #                                         transform_list=train_transform)

    def __init__(self, data_X, data_y, transform_list=None):
        '''
        data_X: numpy array, shape (50000, 3, 32, 32).
        data_Y: python list, shape (50000,)
        
        '''
        self.data = data_X
        self.targets = data_y
        
        self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC

        X_tensor, y_tensor = torch.tensor(self.data), torch.tensor(self.targets)
        
        tensors = (X_tensor, y_tensor)

        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)

        self.tensors = tensors #depends on self.data and self.targets
        self.transforms = transform_list

    def __getitem__(self, index):
     
        img, target = self.data[index], self.targets[index]

        if self.transforms:
          #for transform in self.transforms: 
            img = self.transforms(img)

        # y = self.tensors[1][index]

        return img, target

    def __len__(self):
        return len(self.data) #For a numpy. ndarray , len() returns the size of the first dimension

### class NumpyDataSet2TorchDataset(Dataset)


In [12]:
class NumpyDataSet2TorchDataset(Dataset):
    # adv_train_dataset = CustomTensorDataset(train_concatenated_array, 
    #                                         train_reshaped_labels, 
    #                                         transform_list=train_transform)

    def __init__(self, data_X, data_y, transform_list=None):
        '''
        data_X: numpy array, shape (40000, 32, 32, 3).
        data_Y: numpy array, shape (40000, 10)
        
        '''
        self.data = data_X
        self.targets = np.argmax(data_y, axis=1).tolist() # Return a Python list
              
        self.transforms = transform_list

    def __getitem__(self, index):
     
        img, target = self.data[index], self.targets[index]
       
        # Check the data type of the array
        # Add 0.5 after unnormalizing to [0, 255] to round to nearest integer
        img = np.clip(img * 255 + 0.5, 0, 255).astype(np.uint8)
        
        # doing this so that it is consistent with all other datasets
        # to return a PIL Image                                
        img = Image.fromarray(img)

        if self.transforms:
          #for transform in self.transforms: 
            img = self.transforms(img)

        # y = self.tensors[1][index]

        return img, target

    def __len__(self):
        return len(self.data) 

## Load_dataset

In [13]:
def load_dataset(args):
    '''
    outputs---
    1) dataset: the original dataset before sharing 
    type: torchvision.datasets.cifar.CIFAR10
    2) dataset_test: the global dataset for testing
    type: torchvision.datasets.cifar.CIFAR10 
    3) dg_idx: a list of image indices for sharing 
    type: Python list
    4) dg: the global data sharing dataset
    type: torchvision.datasets.cifar.CIFAR10
    5) dataset_train: the total local data sharing dataset 
    type: torchvision.datasets.cifar.CIFAR10
    6) dict_users: a dictionary of client id - a list of client's example indices
    type: Python dictionary
    '''
    if args.dataset == 'mnist':
        trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
        dataset = datasets.MNIST('../data/mnist/', train=True, download=True, transform=trans_mnist)
        dataset_test = datasets.MNIST('../data/mnist/', train=False, download=True, transform=trans_mnist)
    
    elif args.dataset == 'cifar':
        transform_train = transforms.Compose([
                                            transforms.RandomCrop(32, padding=4),
                                            transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor()])
        
        transform_test = transforms.Compose([ transforms.ToTensor(),
        ])
        dataset = datasets.CIFAR10('../data/cifar', train=True, download=True, 
                                   transform=transform_train)  
        dataset_test = datasets.CIFAR10('../data/cifar', train=False, download=True, 
                                        transform=transform_test)
        print(f"total clean training images: {len(dataset)}")
        print(f"total clean test images: {len(dataset_test)}")
    else:
        exit('Error: unrecognized dataset')
    
    # Make a copy of the original dataset
    dg = copy.deepcopy(dataset)
    dataset_train = copy.deepcopy(dataset)
    
    # selected indices for sharing. The rest for training.
    # dg_idx, dataset_train_idx = train_dg_split(dataset, args)
    dg_idx, dataset_train_idx = np.arange(len(dg)), np.arange(len(dataset_train))
    
    # Pixels based on dg_idx, dataset_train_idx
    dg.data, dataset_train.data = dataset.data[dg_idx], \
                                  dataset.data[dataset_train_idx]
    
    #labels
    if args.dataset == 'cifar':
        dg.targets.clear()
        dataset_train.targets.clear()
        # Labels based on dg_idx
        dg.targets = [dataset[i][1] for i in dg_idx]

        # Labels based on dataset_train_idx   
        dataset_train.targets = [dataset[i][1] for i in dataset_train_idx]
    else:
        dg.targets, dataset_train.targets = dataset.targets[dg_idx], \
                                            dataset.targets[dataset_train_idx]

    # sample users, dataset_train
    if args.sampling == 'iid':
        dict_users = iid(dataset_train, args.num_users)
    elif args.sampling == 'noniid':
        dict_users = noniid(dataset_train, args)
    else:
        exit('Error: unrecognized sampling')
    return dataset, dataset_test, dg_idx, dg, dataset_train, dict_users

In [14]:
# args.num_users = 10 #1 user
# args.sampling= 'iid'

#### Run load_dataset

##### class CIFAR10(VisionDataset)

In [15]:
# import os.path
# import pickle
# from typing import Any, Callable, Optional, Tuple

# import numpy as np
# from PIL import Image

# from torchvision.datasets.utils import check_integrity, download_and_extract_archive
# from torchvision.datasets.vision import VisionDataset


# class MyCIFAR10(VisionDataset):
#     """`CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.

#     Args:
#         root (string): Root directory of dataset where directory
#             ``cifar-10-batches-py`` exists or will be saved to if download is set to True.
#         train (bool, optional): If True, creates dataset from training set, otherwise
#             creates from test set.
#         transform (callable, optional): A function/transform that takes in an PIL image
#             and returns a transformed version. E.g, ``transforms.RandomCrop``
#         target_transform (callable, optional): A function/transform that takes in the
#             target and transforms it.
#         download (bool, optional): If true, downloads the dataset from the internet and
#             puts it in root directory. If dataset is already downloaded, it is not
#             downloaded again.

#     """

#     base_folder = "cifar-10-batches-py"
#     url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
#     filename = "cifar-10-python.tar.gz"
#     tgz_md5 = "c58f30108f718f92721af3b95e74349a"
#     train_list = [
#         ["data_batch_1", "c99cafc152244af753f735de768cd75f"],
#         ["data_batch_2", "d4bba439e000b95fd0a9bffe97cbabec"],
#         ["data_batch_3", "54ebc095f3ab1f0389bbae665268c751"],
#         ["data_batch_4", "634d18415352ddfa80567beed471001a"],
#         ["data_batch_5", "482c414d41f54cd18b22e5b47cb7c3cb"],
#     ]

#     test_list = [
#         ["test_batch", "40351d587109b95175f43aff81a1287e"],
#     ]
#     meta = {
#         "filename": "batches.meta",
#         "key": "label_names",
#         "md5": "5ff9c542aee3614f3951f8cda6e48888",
#     }

#     def __init__(
#         self,
#         root: str,
#         train: bool = True,
#         transform: Optional[Callable] = None,
#         target_transform: Optional[Callable] = None,
#         download: bool = False,
#     ) -> None:

#         super().__init__(root, transform=transform, target_transform=target_transform)

#         self.train = train  # training set or test set

#         if download:
#             self.download()

#         if not self._check_integrity():
#             raise RuntimeError("Dataset not found or corrupted. You can use download=True to download it")

#         if self.train:
#             downloaded_list = self.train_list
#         else:
#             downloaded_list = self.test_list

#         self.data: Any = []
#         self.targets = []

#         # now load the picked numpy arrays
#         for file_name, checksum in downloaded_list:
#             file_path = os.path.join(self.root, self.base_folder, file_name)
#             with open(file_path, "rb") as f:
#                 entry = pickle.load(f, encoding="latin1")
#                 self.data.append(entry["data"])
#                 if "labels" in entry:
#                     self.targets.extend(entry["labels"])
#                 else:
#                     self.targets.extend(entry["fine_labels"])

#         self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
#         self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC

#         self._load_meta()

#     def _load_meta(self) -> None:
#         path = os.path.join(self.root, self.base_folder, self.meta["filename"])
#         if not check_integrity(path, self.meta["md5"]):
#             raise RuntimeError("Dataset metadata file not found or corrupted. You can use download=True to download it")
#         with open(path, "rb") as infile:
#             data = pickle.load(infile, encoding="latin1")
#             self.classes = data[self.meta["key"]]
#         self.class_to_idx = {_class: i for i, _class in enumerate(self.classes)}

#     def __getitem__(self, index: int) -> Tuple[Any, Any]:
#         """
#         Args:
#             index (int): Index

#         Returns:
#             tuple: (image, target) where target is index of the target class.
#         """
#         img, target = self.data[index], self.targets[index]
        
#         # Check the data type of the array
#         print(f'img.shape {img.shape} ')
#         print(f'type(img) {type(img)}')
#         data_type = img.dtype
#         print("Data type:", data_type)
       
        
        
#         # Calculate minimum and maximum values along each channel (axis 2)
#         min_values_per_channel = np.min(img, axis=(0, 1))
#         max_values_per_channel = np.max(img, axis=(0, 1))

#         # Calculate global minimum and maximum values
#         global_min_value = np.min(img)
#         global_max_value = np.max(img)

#         print("Minimum values per channel:", min_values_per_channel)
#         print("Maximum values per channel:", max_values_per_channel)
#         print("Global minimum value:", global_min_value)
#         print("Global maximum value:", global_max_value)
       
        
        
#         # doing this so that it is consistent with all other datasets
#         # to return a PIL Image
#         img = Image.fromarray(img)
        
#         if self.transform is not None:
#             img = self.transform(img)

#         if self.target_transform is not None:
#             target = self.target_transform(target)

#         return img, target

#     def __len__(self) -> int:
#         return len(self.data)

#     def _check_integrity(self) -> bool:
#         for filename, md5 in self.train_list + self.test_list:
#             fpath = os.path.join(self.root, self.base_folder, filename)
#             if not check_integrity(fpath, md5):
#                 return False
#         return True

#     def download(self) -> None:
#         if self._check_integrity():
#             print("Files already downloaded and verified")
#             return
#         download_and_extract_archive(self.url, self.root, filename=self.filename, md5=self.tgz_md5)

#     def extra_repr(self) -> str:
#         split = "Train" if self.train is True else "Test"
#         return f"Split: {split}"

#### Load the clean dataset

In [16]:
args.dataset='cifar'
dataset, dataset_test, dg_idx, dg, dataset_train, dict_users = load_dataset(args)

Files already downloaded and verified
Files already downloaded and verified
total clean training images: 50000
total clean test images: 10000


In [17]:
# share_idx = uniform_distribute(dg, args)
for idx in range(args.num_users):
    print(f'party{idx} local: {len(dict_users[idx])}')
    # print(f'party{idx} share: {len(share_idx)}')
    # local_train_idx = set(list(dict_users[idx]) + share_idx)
    # print(f'party{idx} combined set (no repeat): {len(local_train_idx)}')

party0 local: 10000
party1 local: 10000
party2 local: 10000
party3 local: 10000
party4 local: 10000


# Build model

In [18]:
def build_model(args, img_size):
    """
    Builds a neural network model based on command line arguments and image size.
    Args:
        args (argparse.Namespace): Command line arguments.
        img_size (tuple): Size of the input images.
    Returns:
        A PyTorch model object.
    """
    if args.model == 'cnn' and (args.dataset == 'cifar'):
        net_glob = CNN_v2(args=args).to(args.device)
    elif args.model == 'cnn' and args.dataset == 'mnist':
        net_glob = CNN_v1(args=args).to(args.device)
    elif args.model == 'mlp':
        print('This is a model with two hidden layers')
        len_in = img_size
        net_glob = MLP(dim_in=len_in, dim_hidden=200, dim_out=args.num_classes).to(args.device)
    elif args.model == 'CNN_v2_resume' and args.filepath is not None:
        net_glob = CNN_v2(args=args)
        print('filepath ', args.filepath)
        weights = torch.load(args.filepath)
        net_glob.load_state_dict(weights) 
        net_glob.to(args.device)
    elif args.model == 'customized_resnet18':
        print('This is customized_resnet18')
        net_glob = ResNet18()
        net_glob = net_glob.to(args.device)
        net_glob = torch.nn.DataParallel(net_glob)
        cudnn.benchmark = True
    else:
        net_glob = get_model(args.model)
        for param in net_glob.parameters():
            param.requires_grad = True #fixed feature extractor
        num_ftrs = net_glob.fc.in_features
        net_glob.fc = nn.Linear(num_ftrs, args.num_classes + 1)
        net_glob.to(args.device)
        
    return net_glob

In [19]:
img_size = dataset_train.data[0].shape
img_size

(32, 32, 3)

In [20]:
img_size = dataset_train[0][0].shape
print(img_size)
# build model
net_glob = build_model(args, img_size)

print(net_glob)

torch.Size([3, 32, 32])
This is customized_resnet18
DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3)

In [21]:
trainable_params = sum(
	p.numel() for p in net_glob.parameters() if p.requires_grad
)
print(trainable_params)

11173962


## Gaussian noise (output: 2 Python lists)

In [22]:
def Gaussian_adversarial_images(args, images, mean, std, min_val = 0, max_val = 1):
    '''
    Inputs:
    images: torch.Size([batch size, 3, 32, 32])
    type: Torch.Tensor
    mean: the mean of the Gaussian noise to be added
    std: the standard deviation of the Gaussian noise to be added
    min_val: the minimum value of the images (default: 0)
    max_val: the maximum value of the images (default: 1)
    
    Output:
    images_list: a list of (C x H x W) channel first clean images 
    type: a list of Torch objects
    images_adv_list: a list of (C x H x W) channel first adversarial images 
    type: a list of Torch objects
    '''
    images_adv_list = []
    images_list = []
        
    #images = Variable(images,requires_grad = True)
    noise = torch.normal(mean, std, size=images.shape).cuda()
    
    # add the noise to the images
    images_adv = torch.clamp(images + noise, min=min_val, max=max_val)
    
    images_adv_list.extend(images_adv) #a list of Tensor
    images_list.extend(images) #a list of Tensor
    
    return images_list, images_adv_list

## FGSM (output: 2 Python lists)

In [23]:
def FGSM(args, model, criterion, images, labels, epsilon = 8/255, min_val = 0,max_val = 1):
    '''
    Inputs:
    images: torch.Size([batch size, 3, 32, 32])
    type: Torch.Tensor
    labels: torch.Size([6, 11])
    type: torch.Tensor
    '''
    
    '''
    Output:
    images_list: a list of (C x H x W) channel first clean images 
    type: a list of Torch objects
    images_adv_list: a list of (C x H x W) channel first adversarial images 
    type: a list of Torch objects
    '''
    images_adv_list = []
    images_list = []

    if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
        
        #images = Variable(images,requires_grad = True)
    images.requires_grad = True
    
    outputs = model(images)
    loss =criterion(outputs,labels)

    model.zero_grad()
    if images.grad is not None:
        images.grad.data.fill_(0)
    loss.backward()

    grad = torch.sign(images.grad.data) # Take the sign of the gradient.
    images_adv = torch.clamp(images.data + epsilon*grad,min_val,max_val)     # x_adv = x + epsilon*grad

    # adverserial_images.extend((images_adv).cpu().data.numpy())
    # images_list.extend(images.cpu().data.numpy())
    
    images_adv_list.extend(images_adv) #a list of Tensor
    images_list.extend(images) #a list of Tensor
    
    return images_list, images_adv_list

## FGSM noise

In [24]:
def FGSM_noise(args, model, criterion, images, labels, epsilon = 8/255,
               mean=0, sigma=0.1, min_val = 0, max_val = 1):
        
    images_adv_list = []
    images_list = []
    # with torch.enable_grad():
    if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    images.requires_grad = True

    outputs = model(images)
    loss=criterion(outputs,labels)

    model.zero_grad()
    if images.grad is not None:
        images.grad.data.fill_(0)
    loss.backward()

    grad = torch.sign(images.grad.data) # Take the sign of the gradient.
    images_adv = torch.clamp(images.data + epsilon*grad,min_val,max_val)     # x_adv = x + epsilon*grad
    

    noise = torch.normal(mean, sigma,size=images_adv.shape).to(args.device)
    images_adv_noise = images_adv + noise
    images_adv_noise = torch.clamp(images_adv_noise, min_val, max_val)
    
    images_adv_list.extend(images_adv) #a list of Tensor
    images_list.extend(images) #a list of Tensor
    
    # noise_L2_norm = torch.norm(noise)
    # print(f'noise_L2_norm: {type(noise_L2_norm)}') 
    # print(f'noise_L2_norm: {noise_L2_norm}') 
          
    return images_list, images_adv_list    

## LinfPGDAttack

In [25]:
class LinfPGDAttack(object):
    '''
    Input: tensors
    Output: tensors
    '''
    def __init__(self, model, eps, nb_iter, eps_iter, clip_min, clip_max):
        self.model = model
        self.eps = eps
        self.nb_iter = nb_iter
        self.eps_iter = eps_iter
        self.clip_min = clip_min
        self.clip_max = clip_max


    def perturb(self, x_natural, y):
        x = x_natural.detach()
        x = x + torch.zeros_like(x).uniform_(-self.eps, self.eps)
        for i in range(self.nb_iter):
            x.requires_grad_()
            with torch.enable_grad():
                logits = self.model(x)
                loss = F.cross_entropy(logits, y)
            grad = torch.autograd.grad(loss, [x])[0]
            x = x.detach() + self.eps_iter * torch.sign(grad.detach())
            x = torch.min(torch.max(x, x_natural - self.eps), x_natural + self.eps)
            x = torch.clamp(x, self.clip_min, self.clip_max)
        return x

## visualize_adversarial_images - Long test, remove y_preds

In [26]:
def visualize_adversarial_images(args, round_cur, adversarial_images, y_preds, y_preds_adv, 
                                 images_list, label_list, epsilon):
    if args.dataset == 'cifar' or args.dataset == 'ntga_cifar':
        classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        

        adversarial_images = np.array(adversarial_images)
        y_preds = np.array(y_preds)
        y_preds_adv = np.array(y_preds_adv)
        images_list = np.array(images_list)

        c = adversarial_images - images_list  # Verify whether the max diff between the image and adversarial image is epsilon or not
        if np.any(np.abs(c.max()) > epsilon + 0.01):
            print('the difference is more than the epsilon')

        
        mean = np.array([0.5, 0.5, 0.5])
        mean = mean[:, None, None]
        std = np.array([0.5, 0.5, 0.5])
        std = std[:, None, None]
        

        # Get index of all the images where the attack is successful
        attack = (y_preds != y_preds_adv)
        indexes = np.where(attack == True)[0]

        # Plot the images
        plt_idx = 0
        while plt_idx < 2:
            idx = np.random.choice(indexes)
            img = images_list[idx]
            adv_img = adversarial_images[idx]

            img = img * std + mean
            img = np.transpose(img, (1, 2, 0))
            img = img.clip(0, 1)

            adv_img = adv_img * std + mean
            adv_img = np.transpose(adv_img, (1, 2, 0))
            adv_img = adv_img.clip(0, 1)

            noise = adv_img - img
            noise = np.absolute(10 * noise)  # Noise is multiplied by 10 for visualization purpose
            noise = noise.clip(0, 1)

            if y_preds[idx] != y_preds_adv[idx]:
                disp_im = np.concatenate((img, adv_img, noise), axis=1)
                ax = plt.subplot(1, 2, plt_idx + 1)
                ax.set_title("pred: {}, adv:{}".format(classes[y_preds[idx]], classes[y_preds_adv[idx]]))
                plt.imshow(disp_im)
                plt.xticks([])
                plt.yticks([])
                plt_idx += 1
                print("True Label: ", classes[label_list[idx]], " ", "Predicted Label:", classes[y_preds[idx]], " ",
                      "Adversarial Label:", classes[y_preds_adv[idx]])
        
        name_file = f'./save/{args.fed}_{args.eps_FGSM}_{args.dataset}_{args.model}_{args.local_ep}_nParties_{len(idxs_users)}_{args.sampling}_{args.classwise}_{args.alpha}_round_cur_{round_cur}'
        name_file_1 = name_file + '_visualization.pdf'
        plt.savefig(name_file_1)
        plt.close()

# Test image methods

### calculate_expected_perturbation_proportioncalculate_expected_perturbation_proportion

In [27]:
def calculate_expected_perturbation_proportion(args, adversarial_images, images_list, delta = 1e-10):
    
    adversarial_images = torch.stack(adversarial_images).cpu().detach().numpy()
    # print(adversarial_images.size())
    images_list = torch.stack(images_list).cpu().detach().numpy() #detach
        
    adversarial_images = np.array(adversarial_images)
    images_list = np.array(images_list)
    
    rho = np.zeros(len(images_list)) # record the size of perturbation (2-norm)
    for i in range(len(images_list)):
        diff = images_list[i]-adversarial_images[i]
        
        #L2-norm
        if args.dataset == 'cifar' or args.dataset == 'cifar_cw_3' or args.dataset == 'ntga_cifar':
            diff = diff.reshape((32*32,3))
            
        else:
            diff = diff.reshape((28*28,1))

        rho[i] = LA.norm(diff)/(LA.norm(images_list[i]) + delta)
    #sum
    rho =  np.sum(rho)
    return rho

### Test method 1, no noise

#### DatasetSplit

In [28]:
class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

In [29]:
def test_img(net_g, datatest, idxs, args, criterion):
    net_g.eval()
    
    # testing
    test_loss = 0
    correct = 0
    adv_test_loss = 0
    adv_correct = 0
    adv_correct_2 = 0
    misclassified = 0 #change decision
    
    #visualize
    exp_adv_noise = 0
    y_preds = []
    y_preds_adv = []
    test_images = []
    test_label = []
    adverserial_images = []

    data_loader = DataLoader(DatasetSplit(datatest, idxs), batch_size=args.bs)
    l = len(data_loader)
    for idx, (data, target) in enumerate(data_loader):
        #clean or any data load from the file
        with torch.no_grad():
        
            if args.gpu != -1:
                data, target = data.to(args.device), target.to(args.device) 

            log_probs = net_g(data)
            # sum up batch loss
            # test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
            
            # get the index of the max log-probability
            y_pred = log_probs.data.max(1, keepdim=True)[1]
            correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()
                
        #FGSM method
        images_list, images_adv_list = FGSM(args, net_g, criterion, data, target,
                                                   args.eps_FGSM, args.clip_min, args.clip_max) #The adversarial_images list contains a series of perturbed images
        
        images_adv = torch.stack(images_adv_list) #return a tensor of size (args.bs, 3, 32, 32)
        images_adv = images_adv.to(args.device)
        
        adv_noise = calculate_expected_perturbation_proportion(args, 
                                                               images_adv_list, 
                                                               images_list, 
                                                               1e-10)
        
        with torch.no_grad():
            
            log_probs = net_g(images_adv)
            # sum up batch loss
            # adv_test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
            
            # get the index of the max log-probability
            adv_y_pred = log_probs.data.max(1, keepdim=True)[1]
            adv_correct += adv_y_pred.eq(target.data.view_as(adv_y_pred)).long().cpu().sum()
        
        
        misclassified += (y_pred != adv_y_pred).sum().item()
        y_preds.extend(y_pred.cpu().data.numpy())
        y_preds_adv.extend(adv_y_pred.cpu().data.numpy())
        test_images.extend(data.cpu().data.numpy())
        test_label.extend(target.cpu().data.numpy())
        adverserial_images.extend((images_adv).cpu().data.numpy())
        exp_adv_noise += adv_noise
        
    #test_loss /= len(data_loader.dataset)
    accuracy = 100.00 * correct / len(data_loader.dataset)
    #adv_test_loss /= len(data_loader.dataset)
    adv_acc = 100.00 * adv_correct / len(data_loader.dataset)
    
    #average the noise
    exp_adv_noise = np.sum(exp_adv_noise)/len(data_loader.dataset)
    #visualize
    # visualize_adversarial_images(args, 0, adverserial_images, y_preds, y_preds_adv, 
    #                              test_images, test_label, args.eps_FGSM)
    
    #sum and average adv_noise_proportion
    if args.verbose:
        print('\nTest set: \nAccuracy: {}/{} ({:.2f}%)\n'.format(
                correct, len(data_loader.dataset), accuracy))
        
        
        print('\nFGSM -- Adversarial Test set as a classifier: \nAdv Accuracy: {}/{} ({:.2f}%)\n'.format(adv_correct, len(data_loader.dataset), adv_acc))
        print(f"\nNumber of correct classified clean examples(as compared to clean predictions): {correct}/{len(data_loader.dataset)}")
        print(f"\nNumber of correct classified adversarial examples(as compared to clean predictions): {adv_correct}/{len(data_loader.dataset)}")
        # print(f"\nNumber of attack success: {misclassified}/{len(data_loader.dataset)}")    
    return accuracy, adv_acc, exp_adv_noise

In [30]:
test_idxs = np.arange(100)
data_loader = DataLoader(DatasetSplit(dataset_test, test_idxs), batch_size=args.bs)

In [31]:
loss_fn = torch.nn.CrossEntropyLoss()
test_img(net_glob, dataset_test, test_idxs, args, loss_fn)


Test set: 
Accuracy: 13/100 (13.00%)


FGSM -- Adversarial Test set as a classifier: 
Adv Accuracy: 13/100 (13.00%)


Number of correct classified clean examples(as compared to clean predictions): 13/100

Number of correct classified adversarial examples(as compared to clean predictions): 13/100


(tensor(13.), tensor(13.), 0.06274549971791367)

### Test method 2 with noise

#### evaluate_adversarial_images

In [32]:
def evaluate_adversarial_images(args, images, labels, net_g, criterion, attack_method='Gaussian'):
    
    correct = 0
    if attack_method == 'Gaussian':
    # Generate adversarial images using Gaussian perturbation
       images_list, images_adv_list = Gaussian_adversarial_images(args, images,
                                                                      args.mean, args.sigma, 0, 1)
    
    elif attack_method == 'FGSM':
        adv_correct = 0
        images_list, images_adv_list = FGSM(args, net_g, criterion, images, labels,
                                                       args.eps_FGSM, args.clip_min, args.clip_max)
    elif attack_method == 'FGSM_noise':
        adv_correct = 0
        images_list, images_adv_list = FGSM(args, net_g, criterion, images, labels,
                                                       args.eps_FGSM, args.clip_min, args.clip_max)
        
        adversarial_images = torch.stack(images_adv_list)
        #add noise
        (_), images_adv_list = Gaussian_adversarial_images(args, adversarial_images,
                                                                      args.mean, args.sigma, args.clip_min, args.clip_max)
    else:
        exit('Unrecognized attack method')
        
 
    images_adv = torch.stack(images_adv_list)    
    images_adv = images_adv.to(args.device)
    
    # print(f'images_adv.shape {images_adv.shape}')
    log_probs = net_g(images_adv)
    
    # print(f'log_probs.shape {log_probs.shape}')
    
    adv_y_pred = log_probs.data.max(1, keepdim=True)[1]
    
    # print(f'adv_y_pred.shape {adv_y_pred.shape}')
    
    # print(f'labels {labels.shape}')
    
    correct += adv_y_pred.eq(labels.data.view_as(adv_y_pred)).long().cpu().sum()
    if attack_method != 'Gaussian':
        adv_noise = calculate_expected_perturbation_proportion(args, 
                                                               images_adv_list, 
                                                               images_list, 
                                                               1e-10)
        
        adv_correct += (adv_y_pred == 10).int().cpu().sum()
        #return adversarial attack
        return adv_y_pred, correct, adv_correct, adv_noise
    #return Gaussian noise
    return adv_y_pred, correct
    

#### test evaluate_adversarial_images

In [33]:
# data_loader = DataLoader(dataset_test, batch_size=args.bs)
# criterion = torch.nn.CrossEntropyLoss()
# l = len(data_loader)
#     # Evaluate the classifier on the normal test set 
# for idx, (data, target) in enumerate(data_loader):
#     with torch.no_grad():
#         if args.gpu != -1:
#             data, target = data.to(args.device), target.to(args.device) 

  
#     ######################################### Gaussian noise ##################################
#         Gaussian_pred, Gaussian_correct = evaluate_adversarial_images(args, data,  
#                                                         target,
#                                                         net_glob, 
#                                                         criterion, 
#                                                         'Gaussian')
#         break
        

#### test_img_noise

In [34]:
def test_img_noise(net_g, datatest, idxs, args, criterion):
    net_g.eval()
    
    # testing
    # test_loss = 0
    correct = 0
    FGSM_correct_rand = 0
    
    Gaussian_misclassified = 0 #change decision
    FGSM_rand_misclassified = 0
    
    #visualize
    FGSM_total_adv_noise  = 0
    FGSM_avg_adv_noise = 0.0
    #
    y_preds = []
    Gaussian_pred_list = []
    FGSM_pred_rand_list = []
    #
    test_images = []
    test_label = []
    #
    FGSM_adv_noise = 0
    
    data_loader = DataLoader(DatasetSplit(datatest, idxs), batch_size=args.bs)
    l = len(data_loader)
    # Evaluate the classifier on the normal test set 
    for idx, (data, target) in enumerate(data_loader):
        #clean
        with torch.no_grad():
        
            if args.gpu != -1:
                data, target = data.to(args.device), target.to(args.device) 

            log_probs = net_g(data)
            # sum up batch loss
            # test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
            
            # get the index of the max log-probability
            y_pred = log_probs.data.max(1, keepdim=True)[1]
            correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()
            
    ######################################### Gaussian noise ##################################
            Gaussian_pred, Gaussian_correct = evaluate_adversarial_images(args, data, 
                                                            target, 
                                                            net_g, 
                                                            criterion, 
                                                            'Gaussian')

    ##############################################FGSM method##################################
        
        ##############################################FGSM-noise method##################################
            
        images_list, images_adv_list_noisy = FGSM_noise(args, net_g, criterion, data, 
                                                  target, args.eps_FGSM,
                                                   args.mean, 
                                                  args.sigma, 
                                                  args.clip_min, 
                                                  args.clip_max)
        
        images_adv_noisy = torch.stack(images_adv_list_noisy) #return a tensor of size (args.bs, 3, 32, 32)
        images_adv_noisy = images_adv_noisy.to(args.device)
        
        adv_noise = calculate_expected_perturbation_proportion(args, 
                                                               images_list, 
                                                               images_adv_list_noisy, 
                                                               1e-10)
        with torch.no_grad():
            
            log_probs = net_g(images_adv_noisy)
            # sum up batch loss
            # adv_test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
            
            # get the index of the max log-probability
            FGSM_pred_rand = log_probs.data.max(1, keepdim=True)[1]
            FGSM_correct_rand += FGSM_pred_rand.eq(target.data.view_as(FGSM_pred_rand)).long().cpu().sum()
        
        
        ###############################################################################
        Gaussian_misclassified += (y_pred != Gaussian_pred).sum().item() #proof of vulnerable
        FGSM_rand_misclassified += (y_pred != FGSM_pred_rand).sum().item() #change decision
        
        ###############################################################################
        y_preds.extend(y_pred.cpu().data.numpy())
        Gaussian_pred_list.extend(Gaussian_pred.cpu().data.numpy())
        FGSM_pred_rand_list.extend(FGSM_pred_rand.cpu().data.numpy())
        
        ###############################################################################
        test_images.extend(data.cpu().data.numpy())
        test_label.extend(target.cpu().data.numpy())
        
        FGSM_total_adv_noise += adv_noise
        
    #calculates the accuracy of the model on the clean test examples
    accuracy = 100.00 * correct / len(data_loader.dataset)
    
    #calculates the accuracy of the model on the Gaussian adversarial examples
    Gaussian_acc = 100.00 * Gaussian_correct / len(data_loader.dataset)
        
    #FGSM with noise
    FGSM_acc_noise = 100.00 * FGSM_correct_rand / len(data_loader.dataset)
    
    #average the noise
    FGSM_avg_adv_noise = np.sum(FGSM_total_adv_noise)/len(data_loader.dataset)
    #visualize
    # visualize_adversarial_images(args, 0, adverserial_images, y_preds, y_preds_adv, 
    #                              test_images, test_label, args.eps_FGSM)
    
    #sum and average adv_noise_proportion
    if args.verbose:
        print('\nTest set: \nAccuracy on benign test examples: {}/{} ({:.2f}%)\n'.format(
                correct, len(data_loader.dataset), accuracy))
        
        print('\nTest set: \nAccuracy on Gaussin noise test examples: {}/{} ({:.2f}%)\n'.format(
                Gaussian_correct, len(data_loader.dataset), Gaussian_acc))
        
        print('\nTest accuracy for noisy FGSM: \nAdv Accuracy: {}/{} ({:.2f}%)\n'.format(
                FGSM_acc_noise, len(data_loader.dataset), FGSM_acc_noise))
        
        # print('\nNumber of attack success: {}/{}'.format(FGSM_rand_misclassified, len(data_loader.dataset)))
        #Jing's evaluation
        
    return accuracy, Gaussian_acc, FGSM_acc_noise, FGSM_avg_adv_noise

In [35]:
test_idxs = np.arange(100)
data_loader = DataLoader(DatasetSplit(dataset_test, test_idxs), batch_size=args.bs)

In [36]:
inputs_n, targets= next(iter(data_loader))

print(f'inputs.size {inputs_n.shape}')
print(f'targets.size {targets.shape}')

inputs.size torch.Size([100, 3, 32, 32])
targets.size torch.Size([100])


In [37]:
net_glob(inputs_n) #clear output after being run successfully

tensor([[-1.5260e-02, -4.3604e-03,  1.2958e-02, -7.5203e-03, -2.1702e-02,
          2.7783e-02,  2.6132e-04,  4.6094e-02,  4.9976e-02, -1.5565e-02],
        [-1.4865e-02, -6.2913e-03,  1.2658e-02, -6.0201e-03, -1.7933e-02,
          2.6429e-02,  3.2471e-03,  4.9573e-02,  5.5776e-02, -1.1241e-02],
        [-1.5609e-02, -6.3284e-03,  1.1169e-02, -6.4800e-03, -1.8456e-02,
          2.5335e-02,  2.8501e-03,  4.9339e-02,  5.4343e-02, -1.4442e-02],
        [-1.5799e-02, -7.6395e-03,  1.0399e-02, -5.3582e-03, -1.6178e-02,
          2.3911e-02,  3.5030e-03,  4.9018e-02,  5.6639e-02, -1.3219e-02],
        [-1.4591e-02, -3.2032e-03,  1.2444e-02, -8.7571e-03, -2.1616e-02,
          2.7982e-02, -5.1815e-04,  4.5962e-02,  5.0452e-02, -1.7007e-02],
        [-1.5536e-02, -3.3680e-03,  1.3195e-02, -9.3046e-03, -2.3043e-02,
          2.9431e-02, -6.9858e-04,  4.6389e-02,  4.9353e-02, -1.6242e-02],
        [-1.6097e-02, -3.4330e-03,  1.3570e-02, -7.3728e-03, -2.3982e-02,
          2.9243e-02, -8.5033e-0

# NOTE:  Test only the first 100 images

In [38]:
loss_fn = torch.nn.CrossEntropyLoss()
test_idxs = np.arange(100)
accuracy, Gaussian_acc, FGSM_acc_nois, exp_adv_noise = test_img_noise(net_glob, 
    dataset_test, test_idxs, args, loss_fn)


Test set: 
Accuracy on benign test examples: 13/100 (13.00%)


Test set: 
Accuracy on Gaussin noise test examples: 13/100 (13.00%)


Test accuracy for noisy FGSM: 
Adv Accuracy: 13.0/100 (13.00%)



## generate_adversarial_images

In [39]:
import torch

def generate_adversarial_images(args, images, labels, net_glob, criterion):
    
    # Generate adversarial images using Gaussian perturbation
    images_list, adversarial_images = Gaussian_adversarial_images(args, images,
                                                                  args.mean, args.sigma, 
                                                                  args.clip_min,
                                                                  args.clip_max)
    

    adversarial_images = torch.stack(adversarial_images)
    # print(f'Gaussian noise adversarial_images size: {adversarial_images.shape[0]}')
    
    # Generate adversarial images using Projected Gradient Descent (PGD)
    adversary = LinfPGDAttack(net_glob, eps=args.eps, nb_iter=args.nb_iter, 
                                            eps_iter=args.eps_iter,
                                            clip_min=args.clip_min,
                                            clip_max=args.clip_max)
    
    adversarial_images_PGD = adversary.perturb(images, labels)

    adversarial_images = torch.cat([adversarial_images, adversarial_images_PGD], dim=0)
    # print(f'PGD noise adversarial_images size: {adversarial_images_PGD.shape[0]}')
    
    # print(f'the augmented training set size: {adversarial_images.shape[0]}')

    return adversarial_images


# Utils

### save_model

In [40]:
def save_model(start_time, args, net_glob, idxs_users):
    try:
        now = datetime.now()
        print("Total time for the training: {} seconds ---".format(now - start_time))
        now = start_time.strftime("%Y-%m-%dT%H-%M-%S")

        file = f'{args.fed}_{args.eps_FGSM}_{args.dataset}_{args.model}_{args.rounds}_{args.local_ep}_nParties_{len(idxs_users)}_{args.sampling}__{args.classwise}_{args.alpha}_model'
        model_name = '{}_{}.pt'.format(file, now)
        filepath = './save/'
        
        if not os.path.exists(filepath):
            os.makedirs(filepath)

        f_model = os.path.join(filepath, model_name)
        torch.save(net_glob.state_dict(), f_model)
        
        print('The final model saved to: ', f_model)
        
        return f_model
    except Exception as e:
        print(f'Error saving model: {e}')

### save_model_performance

In [41]:
def save_model_performance(args, start_time, rounds_test_accuracy,
                           rounds_train_loss, 
                           rounds_adv_test_accuracy, 
                           rounds_adv_test_accuracy_2,
                           idxs_users):
    try:
        output = {}
        output['rounds_test_accuracy'] = rounds_test_accuracy
        output['rounds_train_loss'] = rounds_train_loss
        output['rounds_adv_test_accuracy'] = rounds_adv_test_accuracy
        output['rounds_adv_test_accuracy_2'] = rounds_adv_test_accuracy_2

        temp='./save/'
        

        filename = f'{args.fed}_{args.eps_FGSM}_{args.dataset}_{args.model}_${args.rounds}_{args.local_ep}_nParties_{len(idxs_users)}_{args.sampling}_{args.classwise}_{args.alpha}'
        filename = '{}_{}.out'.format(filename, start_time.strftime("%Y-%m-%dT%H-%M-%S"))
        filepath = os.path.join(temp, filename)
        print('filepath ', filepath)
        outfile = open(filepath,'wb')
        pickle.dump(output, outfile)

        print('The output file saved to: ', filepath)
    except Exception as e:
            print(f'Error saving model performance: {e}')

### read_model_performance

In [42]:
def read_model_performance(args, temp, idxs_users):
    try:
        temp='./save/'
        filename = f'{args.fed}_{args.eps_FGSM}_{args.dataset}_{args.model}_{args.local_ep}_nParties_{idxs_users}_{args.sampling}_{args.classwise}_{args.alpha}_output.out'
        filepath = os.path.join(temp, filename)
        print('filepath ', filepath)
        
        output = pickle.load(open(filepath, "rb"))
        
        rounds_test_accuracy = output['rounds_test_accuracy']
        rounds_train_loss = output['rounds_train_loss']
        rounds_adv_test_accuracy = output['rounds_adv_test_accuracy'] 
        rounds_adv_test_accuracy_2 = output['rounds_adv_test_accuracy_2']
        
        print(len(rounds_test_accuracy))
        print(len(rounds_adv_test_accuracy_2))
        print(len(rounds_train_loss))
        ################### training loss ######################
#         label = args.fed
#         plt.xlabel("Global Iterations")
#         plt.ylabel(f"Train loss")
#         # plt.set_yscale('log')
#         plt.title(f"Centralized AT - Dataset: {args.dataset} - Model: {args.model} - Soft labelling" )
#         plt.plot(rounds_train_loss, label='soft label')
        
#         plt.legend(loc="center right")
#         plt.show()
#         plt.close()
        ###################################################
        label = args.fed
        plt.xlabel("Global Iterations")
        plt.ylabel(f"Test acc")
        # plt.set_yscale('log')
        plt.title(f"Centralized AT - Dataset: {args.dataset} - Model: {args.model} - Soft labelling" )
        plt.plot(rounds_test_accuracy, label='clean acc')
        plt.plot(rounds_adv_test_accuracy, label='additional class acc')
        plt.plot(rounds_adv_test_accuracy_2, label='robust acc')
        
        plt.legend(loc="lower right")
        plt.show()
        plt.close()
        
    except Exception as e:
            print(f'Error loading model performance: {e}')

### parse_output_log

In [43]:
import re

def parse_output_log(args, file_name, save_file_path):
    train_loss = []
    test_accuracy = []
    adv_accuracy = []
    
    for line in open(file_name, 'r'):
        # print(line) 
        # Search for average train loss over parties
        search_avg_train_loss = re.search(r'Average train loss over \[(.*)\] parties: ([\d\.]+)', line, re.M|re.I)
        if search_avg_train_loss:
            val = float(search_avg_train_loss.group(2))
            train_loss.append(val/args.num_users)
        
        # # search for test accuracy
        # search_test_accu = re.search(r'Number of correct classified clean examples\(as compared to clean predictions\): (\d+)/\d+', line, re.M|re.I)
        # if search_test_accu:
        #     val = float(search_test_accu.group(1))
        #     # print(val)
        #     test_accuracy.append(val/100)
        #     # if(len(test_accuracy) == 5):
        #     #     break
        # # search for adversarial accuracy
        # search_adv_accu = re.search(r'Adv Accuracy: \d+/\d+ \((\d+\.\d+)%\)', line, re.M|re.I)
        # if search_adv_accu:
        #     val = float(search_adv_accu.group(1))
        #     if val > 1.0:
        #         adv_accuracy.append(float(search_adv_accu.group(1)))
        
    print(f'train_loss {len(train_loss)}')
    # print(f'test_accuracy {len(test_accuracy)}')
    # print(f'adv_accuracy {len(adv_accuracy)}')
    
    # plot train loss
    plt.figure(figsize=(8, 4))
    plt.xlabel("Global Iterations")
    plt.ylabel(f"Train loss")

    filename = f'{args.fed}_{args.adv_eps}_{args.dataset}_{args.model}_{args.local_ep}_nParties_{args.num_users}_{args.sampling}_{args.classwise}_{args.alpha}'

    plt.title(f"AT - Dataset: {args.dataset} - Model: {args.model} - N = {args.num_users} - Sampling: {args.sampling}" )
    plt.plot(train_loss, label='train loss')
    plt.legend(loc="lower right")
    filepath_1 = save_file_path + '/' + f'{filename}_train_loss.png'
    print(filepath_1)
    plt.savefig(filepath_1)
    # plt.show()
    plt.close()
    
    # # plot test accuracy
    # plt.xlabel("Global Iterations")
    # plt.ylabel(f"Test acc")
    # # plt.set_yscale('log')
    # plt.title(f"AT - Dataset: {args.dataset} - Model: {args.model} - N = {args.num_users} - Sampling: {args.sampling}" )
    # plt.plot(test_accuracy, label='clean acc') #scale: 0.8
    # plt.plot(adv_accuracy, label='robust acc') #percentage

    # plt.legend(loc="lower right")
    # # plt.show()
    # print(filepath_2)
    # filepath_2 = save_file_path + '/' + f'{filename}_robust_acc.png'
    # plt.savefig(filepath_2)
    # plt.close()

### Run parse_output_log

In [44]:
# file_name = 'AT_out_fedavg_0.031_cifar_customized_resnet18_100_3_nParties_twoclassnoniid_10_noshare'
# parse_output_log(args, file_name, './save/AT/Figures')

# Update local weights using adversarial examples and soft labels

In [45]:
def adjust_learning_rate(args, optimizer, epoch):
        lr = args.lr
        if epoch >= 100:
            lr /= 10
        if epoch >= 150:
            lr /= 10
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

### ModelUpdate

In [46]:
class ModelUpdate(object):
    def __init__(self, args, client_id, local_ep=1, dataset=None, idxs=None):
        self.args = args
        self.loss_func = nn.CrossEntropyLoss()
        self.ldr_train = DataLoader(DatasetSplit(dataset, idxs), 
                                    batch_size=self.args.local_bs, shuffle=True)
        self.local_ep = local_ep
        self.client_id = client_id
        
    def train(self, pid, round_cur, local_net, net):
        
        net.train()
        
        # train and update
        # weight_decay=self.args.l2_lambda
        if self.args.l2_lambda != 0:
            optimizer = torch.optim.SGD(net.parameters(), lr=self.args.lr, 
                                        weight_decay=self.args.l2_lambda,
                                        momentum=self.args.momentum)
        else:
            optimizer = torch.optim.SGD(net.parameters(), lr=self.args.lr, 
                                        momentum=self.args.momentum)
        epoch_loss = []

        if self.args.sys_homo: 
            local_ep = self.local_ep
        else:
            local_ep = randint(self.args.min_le, self.args.max_le) 
        
        for iter in range(local_ep):
            print(f'\n[ Current round {round_cur} Train epoch: {iter} ]')
            batch_loss = []
            correct = 0
            train_size = 0

            # adjust_learning_rate(self.args, optimizer, iter)
            
            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                
                #C&W
                images, labels = images.to(self.args.device), labels.to(self.args.device)
                net.zero_grad()
                log_probs = net(images) 
                loss = self.loss_func(log_probs, labels) 
                    
                loss.backward()
                
                optimizer.step()
                
                # get the index of the max log-probability
                y_pred = log_probs.data.max(1, keepdim=True)[1]
                correct += y_pred.eq(labels.data.view_as(y_pred)).long().cpu().sum()
                train_size += images.shape[0]
                if self.args.verbose and batch_idx % 10 == 0:
                    print('Round : {} Party {}: Update Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        round_cur, pid, iter, batch_idx * len(images), len(self.ldr_train.dataset),
                               100. * batch_idx / len(self.ldr_train), loss.item()))
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))
            print('\nTotal adversarial train accuarcy:', 100. * correct / train_size)
            print('\nTotal adversarial train loss:', sum(batch_loss)/len(batch_loss))
           
        return net.state_dict(), sum(epoch_loss) / len(epoch_loss)

### Adversarial_ModelUpdateAdversarial_ModelUpdate

- generate_adversarial_images
- modify the label
- calculate the loss
- Take the first derivative of the loss func wrt to the model weights.


In [47]:
class Adversarial_ModelUpdate(object):
    def __init__(self, args, client_id, local_ep=1, dataset=None, idxs=None):
        self.args = args
        self.loss_func = nn.CrossEntropyLoss()
        self.ldr_train = DataLoader(DatasetSplit(dataset, idxs), 
                                    batch_size=self.args.local_bs, shuffle=True)
        self.local_ep = local_ep
        self.client_id = client_id
        
    
    def smooth_label(self, y):
        '''
        y: integer-base
        '''
        # convert y to one-hot encoding
        y_onehot = F.one_hot(y, self.args.num_classes)
        # smooth the one-hot encoding
        y_smooth = (self.args.soft_label_clean) * y_onehot + (1 - self.args.soft_label_clean) / (self.args.num_classes)
        # convert back to class labels
        y_smooth = torch.argmax(y_smooth, dim=1)
        return y_smooth

    def train(self, pid, round_cur, local_net, net):
        
        net.train()
        
        # train and update
        # weight_decay=self.args.l2_lambda
        if self.args.l2_lambda != 0:
            optimizer = torch.optim.SGD(net.parameters(), lr=self.args.lr, weight_decay=self.args.l2_lambda, momentum=self.args.momentum)
        else:
            optimizer = torch.optim.SGD(net.parameters(), lr=self.args.lr, momentum=self.args.momentum)
        epoch_loss = []

        if self.args.sys_homo: 
            local_ep = self.local_ep
        else:
            local_ep = randint(self.args.min_le, self.args.max_le) 
        
        for iter in range(local_ep):
            print(f'\n[ Current round {round_cur} Train epoch: {iter} ]')
            batch_loss = []
            correct = 0
            train_size = 0

            adjust_learning_rate(self.args, optimizer, iter)
            
            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                
                #C&W
                images, labels = images.to(self.args.device), labels.to(self.args.device)

                adversarial_images = generate_adversarial_images(self.args, 
                                                                    images, 
                                                                    labels, 
                                                                    net, 
                                                                    self.loss_func)
    
                adversarial_images = adversarial_images.to(self.args.device)
    
                labels_smooth = self.smooth_label(labels) #Test on Gaivi
                
                soft_label_list = torch.cat([labels_smooth, labels_smooth], dim=0)

                soft_label_list = soft_label_list.to(self.args.device)

                net.zero_grad()
                log_probs = net(adversarial_images) #model predictions, no torch concaternation
                loss = self.loss_func(log_probs, soft_label_list) #labels here 11 class and clean examples
                    
                loss.backward()
                
                optimizer.step()
                
                # get the index of the max log-probability
                y_pred = log_probs.data.max(1, keepdim=True)[1]
                correct += y_pred.eq(soft_label_list.data.view_as(y_pred)).long().cpu().sum()
                train_size += adversarial_images.shape[0]
                if self.args.verbose and batch_idx % 10 == 0:
                    print('Round : {} Party {}: Update Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        round_cur, pid, iter, batch_idx * len(images), len(self.ldr_train.dataset),
                               100. * batch_idx / len(self.ldr_train), loss.item()))
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))
            print('\nTotal adversarial train accuarcy:', 100. * correct / train_size)
            print('\nTotal adversarial train loss:', sum(batch_loss)/len(batch_loss))
           
        return net.state_dict(), sum(epoch_loss) / len(epoch_loss)

# Update global weights

#### FedAvg

In [48]:
def FedAvg(w, args, c_global=None, res_caches=None):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        tmp = torch.zeros_like(w[0][k], dtype = torch.float32).to(args.device)
        for i in range(len(w)):
            tmp += w[i][k]
        tmp = torch.true_divide(tmp, len(w))
        w_avg[k].copy_(tmp)
    if args.fed == 'scaffold' and c_global is not None and res_caches is not None:
        if args.all_clients:
            client_num_per_round = args.num_users
        else:
            client_num_per_round = max(int(args.frac * args.num_users), 1)
        
        # update global model
        avg_weight = torch.tensor(
                        [
                            1 / client_num_per_round
                            for _ in range(client_num_per_round)
                        ],
                        device=args.device,
                    ) #by number of selected clients per round, not dependent on the local data size
        # y_pred = net_glob(inputs).cpu()
        # print(y_pred.detach().numpy())
        # print(f'avg_weight: {avg_weight.cpu().detach().numpy()}')
        c_delta_cache = list(zip(*res_caches))
        # print(f'c_delta_cache {c_delta_cache}')
        # update global control
        for c_g, c_del in zip(c_global, c_delta_cache):
            # print(f'before c_g {c_g.cpu().detach().numpy()}')
            c_del = torch.sum(avg_weight * torch.stack(c_del, dim=-1), dim=-1) #delta_c = sum of avg_weight * delta_c_i
            # print(f'c_del: {c_del.cpu().detach().numpy()}')
            c_g.data += (
                client_num_per_round / args.num_users
            ) * c_del #c_global = |S| / N * c_delta
            # print(f'c_g {c_g.cpu().detach().numpy()}')
        return w_avg, c_global
    return w_avg

#### get_scores

In [49]:
def get_scores(distance, th):
    """
    Sorts the distances in an ordered list 
    and returns the list for use to 
    the fusion_collected_responses function

    :param distance: List of distance vector
    :type distance: `list`
    :param th: Threshold
    :return: list of summation of distances
    :rtype: `list`
    """
    distance.sort(axis=1)
    
    # print(f'sorted distance {distance}')
    # the +1 is added to account for the zero entry (distance from itself)
    return np.sum(distance[:, 0:th+1], axis=1)

#### Util_ModelUpdate

In [50]:
from pickle import loads, dumps, dump
class Util_ModelUpdate():
    """
    Class to hold model update dictionary. Dictionary can only be accessed
    using inbuild methods like `get` and `add`
    """

    def __init__(self, **kwargs):
        """
        Initialize the dictionary and add updates.
        :param kwargs: Dictionary of model-update specific arguments.
        :type kwargs: `dict`
        """
        self.__updates = {}
        for key, value in kwargs.items():
            # print(key)
            # print(type(value))
            self.add(key, value)

    def add(self, key, value):
        """
        Add update `value` for `key`

        :param key: Identifier which represents the update
        :type key: `str`
        :param value: Content of the update
        :type value: any ds/object which can be pickled
        """
        try:
            self.__updates[key] = dumps(value)

        except Exception as ex:
            logger.exception(ex)

            logger.exception("Error occured while adding a update.\
                                Make sure model update data structure is picklable")

            raise ModelUpdateException("Error updating model update")

    def get(self, key):
        """
        Get update value for given key from model update dictionary

        :param key: Identifier which represents the update
        :type key: `str`
        :return: content of the update
        :rtype: any ds/object after its unpickled
        """
        if key not in self.__updates:
            logger.error("Key "+key+" not found in model updates")
            raise ModelUpdateException(
                "Invalid key was requested from model update")

        ret_val = loads(self.__updates[key])
        return ret_val

    def exist_key(self, key):
        """
        Check if a specified key is used in model update dictionary

        :param key: Identifier which represents the update
        :type key: `str`
        :return: check result of existing the key
        :rtype: True or False
        """
        return True if key in self.__updates else False

#### flatten_model_update

In [51]:
def flatten_model_update(w_local):
    """
    Generates a flattened np array for all of the layerwise weights of an update

    :param lst_layerwise_wts: List of layer weights
    :type lst_layerwise_wts: `list`
    :return: `np.array`
    """
    lst_layerwise_wts = Util_ModelUpdate(weights=w_local).get('weights') #neural network and bias parameters
    
    # print(f'lst_layerwise_wts {type(lst_layerwise_wts)}')         
    # print(f'lst_layerwise_wts {lst_layerwise_wts}')
    wt_vector = []
    for w in lst_layerwise_wts:
        # print(f'w: type {type(w)} value {w}')
        # t = w.flatten()
        t = lst_layerwise_wts[w].flatten().cpu()
        wt_vector = np.concatenate([wt_vector, t])
    print(f'{wt_vector}\n')
    return wt_vector

#### Krum

In [52]:
def Krum(lst_model_updates, args, c_global=None, res_caches=None):
    """
    Sorts the distances in an ordered list 
    and returns the list for use to 
    the fusion_collected_responses function

    - w: A list containing model updates from different clients.
    - args: A configuration object or dictionary containing various settings.
    - c_global: Global model parameters (optional).
    - res_caches: Cached values (optional).
    :return: the selected model
    :type: `nn.Module`
    """
    num_updates = len(lst_model_updates)
    distance = np.zeros((num_updates, num_updates), dtype=float)

    lst_model_updates_flattened = []
    for i in range(len(lst_model_updates)):
        lst_model_updates_flattened.append(flatten_model_update(lst_model_updates[i]))

    #Build distance matrix
    for i in range(num_updates):
        curr_vector = lst_model_updates_flattened[i]
        for j in range(num_updates):
            if j is not i:
                distance[i, j] = \
                    np.square(
                        np.linalg.norm(
                        curr_vector - lst_model_updates_flattened[j]) # Default is L-2 norm
                    ) #square distance 

    th = num_updates - args.byzantine_threshold - 2 #th means threshold, benign clients                
    scores = get_scores(distance, th)
    selected_idx = np.argmin(scores)
    selected_net = lst_model_updates[selected_idx]
    
    # Print the 'distance' matrix
    print("\nDistance Matrix:")
    for row in distance:
        print(row)
    
    print(f"scores {scores}")
    print(f"selected_idx {selected_idx}")
    
    return selected_net

In [53]:
# v1 = [0.3848, 0.2165, 0.179, -0.21, 0.2628]

In [54]:
# v2 = [-0.4749, 0.3494, -0.6478,-0.5333, -0.8635]

In [55]:
# square L2-distance 
# np.square(
#                 np.linalg.norm(np.array(v1) - np.array(v2))
#           ) 

#### Test Krum with GPU

In [56]:
# Define your linear regression models
num_models = 5

input_size = 2
hidden_size = 1
output_size = 1

# Create a list of linear regression models (mocked with random parameters)
lst_model_updates = []
for i in range(num_models):   
    net =  MLP(dim_in=input_size, dim_hidden=hidden_size, dim_out=output_size).to(args.device)
    state_dict =  net.state_dict() #current weights and a dictionary object
    lst_model_updates.append(state_dict)
    print(f'Model {i}\n')
    for k in state_dict:
        print(k, "\t", state_dict[k])


Model 0

layer_input.weight 	 tensor([[ 0.2947, -0.3736]], device='cuda:0')
layer_input.bias 	 tensor([0.1801], device='cuda:0')
layer_hidden.weight 	 tensor([[0.6136]], device='cuda:0')
layer_hidden.bias 	 tensor([-0.2590], device='cuda:0')
Model 1

layer_input.weight 	 tensor([[0.3409, 0.3530]], device='cuda:0')
layer_input.bias 	 tensor([0.3236], device='cuda:0')
layer_hidden.weight 	 tensor([[0.6977]], device='cuda:0')
layer_hidden.bias 	 tensor([-0.5504], device='cuda:0')
Model 2

layer_input.weight 	 tensor([[0.4603, 0.2358]], device='cuda:0')
layer_input.bias 	 tensor([0.1099], device='cuda:0')
layer_hidden.weight 	 tensor([[0.3144]], device='cuda:0')
layer_hidden.bias 	 tensor([0.0370], device='cuda:0')
Model 3

layer_input.weight 	 tensor([[0.5939, 0.5582]], device='cuda:0')
layer_input.bias 	 tensor([0.3992], device='cuda:0')
layer_hidden.weight 	 tensor([[0.6112]], device='cuda:0')
layer_hidden.bias 	 tensor([-0.9343], device='cuda:0')
Model 4

layer_input.weight 	 tensor([[

In [57]:
# Test the Krum function
results = Krum(lst_model_updates, args)
print(type(results))

[ 0.29473367 -0.373568    0.18013968  0.61362445 -0.25898147]

[ 0.34085208  0.35295913  0.32360089  0.69771278 -0.55036688]

[0.46034127 0.23579425 0.1099237  0.31443894 0.03700936]

[ 0.59390533  0.5581997   0.39917818  0.61123538 -0.93429756]

[-0.2716195   0.2518141   0.08424059 -0.35886574 -0.61117744]


Distance Matrix:
[0.         0.58080105 0.64252599 1.46173013 1.79083449]
[0.         0.26675268 0.5655729  0.64252599 1.56270119]
[0.         0.5655729  0.58080105 1.23697805 1.41016815]
[0.         0.26675268 1.23697805 1.46173013 1.98769384]
[0.         1.41016815 1.56270119 1.79083449 1.98769384]
scores [0.58080105 0.26675268 0.5655729  0.26675268 1.41016815]
selected_idx 1
<class 'collections.OrderedDict'>


In [58]:
global_net =  MLP(dim_in=input_size, dim_hidden=hidden_size, dim_out=output_size)

global_net.load_state_dict(results, strict=True)

print("global_net Weights:")
state_dict =  global_net.state_dict() #current weights and a dictionary object
for k in state_dict:
    print(k, "\t", state_dict[k])

global_net Weights:
layer_input.weight 	 tensor([[0.3409, 0.3530]])
layer_input.bias 	 tensor([0.3236])
layer_hidden.weight 	 tensor([[0.6977]])
layer_hidden.bias 	 tensor([-0.5504])


#### CoordinateMedian

In [59]:
def CoordinateMedian(lst_model_updates, args, c_global=None, res_caches=None):
    """
    the averaging aggregation is performed using Coordinate-Median policy model weights.
    Implements the algorithm in Byzantine-Robust Distributed Learning: 
    Towards Optimal Statistical Rates: https://arxiv.org/pdf/1803.01498.pdf

    - w: A list containing model updates from different clients.
    - args: A configuration object or dictionary containing various settings.
    - c_global: Global model parameters (optional).
    - res_caches: Cached values (optional).
    :return: results after aggregation
    :type: `list`
    """
    parameters = []
    results = []
    
    for i in range(len(lst_model_updates)):
        w_local = lst_model_updates[i]
        lst_layerwise_wts = Util_ModelUpdate(weights=w_local).get('weights') #list of layer weights
        # print(f'Model {i} {lst_layerwise_wts}')
        parameters.append(lst_layerwise_wts)
    
    # # print(f'parameters {parameters}\n')
    # for layer in zip(*parameters):
    #     print(f' layer {type(layer)} layer {layer}')
        
    for layer in zip(*parameters):
        # print(f' layer {layer}')
        
        tensors = []
        temp = []
        for i, tensor in enumerate(layer):
            # print(f'tensor {tensor}')
            tensors.append(parameters[i][tensor])
            temp.append(np.array(parameters[i][tensor].cpu()))
        
        # print(f'list of tensors {tensors}\n')
        # print(f'layer {layer} the corresponding numpy array {temp}\n')
        
        results.append(np.median(temp, axis=0))
    
    # print(f'results {results}\n')
        
    return results

#### Test CoordinateMedian

In [60]:
# Test the CoordinateMedian function
results = CoordinateMedian(lst_model_updates, args)


In [61]:
#Load the model using the results
global_net =  MLP(dim_in=input_size, dim_hidden=hidden_size, dim_out=output_size)
state_dict =  global_net.state_dict() #current weights and a dictionary object

for k, v in zip(state_dict.keys(), results):
    v = torch.from_numpy(v) #Create tensors
    v = v.to(args.device)
    v.requires_grad = True
    state_dict[k] = v
global_net.load_state_dict(state_dict, strict=True)

print("global_net Weights:")
state_dict =  global_net.state_dict() #current weights and a dictionary object
for k in state_dict:
    print(k, "\t", state_dict[k])

global_net Weights:
layer_input.weight 	 tensor([[0.3409, 0.2518]])
layer_input.bias 	 tensor([0.1801])
layer_hidden.weight 	 tensor([[0.6112]])
layer_hidden.bias 	 tensor([-0.5504])


## Initialization using Data Sharing

In [62]:
net_glob.train()

# copy weights
w_glob = net_glob.state_dict()
        
# # initialization stage of FedShare
# print('initialization stage of FedShare')
# #share a warm-up model trained on the global dataset, dg_idx,
# initialization_stage = ModelUpdate(args=args, 
#                                   client_id=9999,
#                                    local_ep=args.local_ep, 
#                                    dataset=dataset, 
#                                    idxs=set(dg_idx))
# w_glob, _ = initialization_stage.train(pid=999, 
#                                        round_cur=9999, 
#                                        local_net = copy.deepcopy(net_glob).to(args.device), 
#                                        net = copy.deepcopy(net_glob).to(args.device))

## Test image

### Method 1

In [63]:
net_glob.load_state_dict(w_glob)
loss_fn = torch.nn.CrossEntropyLoss()
###
test_idxs = np.arange(1000)
###
acc_test, adv_acc_test,(_) = test_img(net_glob, dataset_test, 
                                                       test_idxs,
                                                       args, loss_fn)



Test set: 
Accuracy: 106/1000 (10.60%)


FGSM -- Adversarial Test set as a classifier: 
Adv Accuracy: 97/1000 (9.70%)


Number of correct classified clean examples(as compared to clean predictions): 106/1000

Number of correct classified adversarial examples(as compared to clean predictions): 97/1000


### Method 2

In [64]:
loss_fn = torch.nn.CrossEntropyLoss()
###
test_idxs = np.arange(1000)
###
accuracy, Gaussian_acc, FGSM_acc_noise, (_) = test_img_noise(net_glob, dataset_test,
                                                        test_idxs,
                                                        args, loss_fn)


Test set: 
Accuracy on benign test examples: 106/1000 (10.60%)


Test set: 
Accuracy on Gaussin noise test examples: 14/1000 (1.40%)


Test accuracy for noisy FGSM: 
Adv Accuracy: 9.699999809265137/1000 (9.70%)



## Local data partition

In [65]:
dict_users.keys()

dict_keys([0, 1, 2, 3, 4])

### uniform_distribute

In [66]:
def uniform_distribute(dataset, args): 
    globally_shared_data_idx = []
    
    idxs = np.arange(len(dataset))
    
    if args.dataset == "mnist":
        labels = dataset.targets.numpy()
    elif args.dataset == "cifar" or args.dataset == "adv_cifar" or args.dataset == "cifar_cw_3" or args.dataset == "ntga_cifar" :
        labels = np.array(dataset.targets)
    else:
        exit('uniform_distribute - Error: unrecognized dataset')
    
    idxs_labels = np.vstack((idxs, labels))
    idxs_labels = idxs_labels[:,idxs_labels[1,:].argsort()]

    idxs = idxs_labels[0]
    labels = idxs_labels[1]
    
    for i in range(args.num_classes):
        specific_class = np.extract(labels == i, idxs)
        globally_shared_data = np.random.choice(specific_class, int(args.alpha * args.classwise), replace=False)
        
        globally_shared_data_idx = globally_shared_data_idx + list(globally_shared_data)
    
    return globally_shared_data_idx

In [67]:
#### dict_users

In [68]:
# share_idx = uniform_distribute(dg, args)
for idx in range(args.num_users):
    print(f'party{idx} local: {len(dict_users[idx])}')
    # print(f'party{idx} share: {len(share_idx)}')
    # local_train_idx = set(list(dict_users[idx]) + share_idx)
    # print(f'party{idx} combined set (no repeat): {len(local_train_idx)}')

party0 local: 10000
party1 local: 10000
party2 local: 10000
party3 local: 10000
party4 local: 10000


# Main training loop

In [69]:
start_time = datetime.now()
img_size = dataset_train[0][0].shape
# build model
# net_glob = build_model(args, img_size)
w_glob = net_glob.state_dict() #w_glob
# print(f'w_glob {w_glob}')
if args.all_clients: 
    print("Aggregation over all clients")
    w_locals = [w_glob for i in range(args.num_users)]
    rounds_train_loss = {user: list() for user in range(args.num_users)}

#define the loss function
criterion = torch.nn.CrossEntropyLoss()

train_loss = []
rounds_test_accuracy = []
rounds_adv_test_accuracy = []
rounds_adv_test_accuracy_2 = []

for iter in trange(args.rounds): 
        
    if not args.all_clients:
        w_locals = []
    
    rounds_train_loss = []

    # Calculate n and m
    n = max(int(args.frac * args.num_users), 1)
    m = args.byzantine_threshold

    # Generate all possible indices
    all_indices = list(range(n))

    # Generate attacker indices
    attacker_list = np.random.choice(all_indices, m, replace=False)


    # Calculate non-attacker indices
    idxs_users = list(set(all_indices) - set(attacker_list))

    print("Attacker Indices:", attacker_list)
    print("Non-Attacker Indices:", idxs_users)

    
    for idx in idxs_users:
        # Local update
        local = ModelUpdate(args=args,
                            client_id=idx,
                            local_ep=args.local_ep, 
                            dataset=dataset, 
                            idxs=set(list(dict_users[idx])))
        #On each client, compute mini-batch gradient and control variate   
        w, loss = local.train(pid=idx, 
                            round_cur=iter, 
                            local_net = copy.deepcopy(net_glob).to(args.device), 
                            net = copy.deepcopy(net_glob).to(args.device))

        rounds_train_loss.append(loss) #client level
        #Save local model
        if args.all_clients:
            w_locals[idx] = copy.deepcopy(w)
        else:
            w_locals.append(copy.deepcopy(w))

    for idx in attacker_list:
        # Local update
        local = ModelUpdate(args=args,
                            client_id=idx,
                            local_ep=args.local_ep, 
                            dataset=dataset, 
                            idxs=set(list(dict_users[idx]))) #Different data
        #On each client, compute mini-batch gradient and control variate   
        w, loss = local.train(pid=idx, 
                            round_cur=iter, 
                            local_net = copy.deepcopy(net_glob).to(args.device), 
                            net = copy.deepcopy(net_glob).to(args.device))

        rounds_train_loss.append(loss) #client level
        #Save local model
        if args.all_clients:
            w_locals[idx] = copy.deepcopy(w)
        else:
            w_locals.append(copy.deepcopy(w))
        
    #End on clients
    if args.fed == 'scaffold':
        w_glob, c_global = FedAvg(w_locals, args, c_global, res_caches) #update at each round
    elif args.fed == 'Krum':
        w_glob = Krum(w_locals, args)
        
    elif args.fed == 'CoordinateMedian':
        results = CoordinateMedian(w_locals, args)
        #manual loading. Prone to issues.
        for k, v in zip(w_glob.keys(), results):
            # print(f'type v {v}')
            if isinstance(v, float):
                # print(f'median {v}')
                # print(f'w_glob[k] {w_glob[k]}')
                pass
            else:
                v = torch.from_numpy(v) #Create tensors
                v = v.to(args.device)
                v.requires_grad = True
                w_glob[k] = v
    else:
        w_glob = FedAvg(w_locals, args)
    
    # try replicate krum with mnist and federated average, using the paper experiment. 
    # try krum and cifar-10

    # copy weight to net_glob
    net_glob.load_state_dict(w_glob, strict=True)
    
    # acc_test, loss_test, adv_acc_test, adv_loss_test = test_img(net_glob, dataset_test, args, criterion)
   
    train_loss.append(sum(rounds_train_loss))
    
    acc_test, adv_acc_test, (_) = test_img(net_glob, 
                                                                     dataset_test, 
                                                                     test_idxs,
                                                                     args, 
                                                                     criterion)
    
    accuracy, Gaussian_acc, FGSM_acc_noise, exp_adv_noise = test_img_noise(net_glob, 
                                                            dataset_test,
                                                            test_idxs,
                                                            args, 
                                                            criterion)
    
    rounds_test_accuracy.append(acc_test)
    rounds_adv_test_accuracy.append(adv_acc_test)
    rounds_adv_test_accuracy_2.append(FGSM_acc_noise)
    
    if args.debug:
        print(f"Round: {iter}")
        print(f"Average train loss over {rounds_train_loss} parties: {sum(rounds_train_loss)}")
        print(f"Test accuracy: {acc_test}")
        print(f"Adv Test accuracy: {adv_acc_test}")
        print(f"Adv Test accuracy With Noise: {FGSM_acc_noise}")
    
   

    f_model_path = save_model(start_time, args, net_glob, idxs_users)
    print('Model Saved!')
    
     #stopping criteria here. 
    if exp_adv_noise > args.rho:
        print(f'exp_adv_noise {exp_adv_noise}')
        print("A larger perturbation is required to fool the classifier. Thus, the classifier is more robust.")
        break
    save_model_performance(args, start_time, rounds_test_accuracy, 
                               train_loss, 
                               rounds_adv_test_accuracy,
                           rounds_adv_test_accuracy_2,
                               idxs_users)



Aggregation over all clients


  0%|          | 0/2 [00:00<?, ?it/s]

Attacker Indices: [1 2]
Non-Attacker Indices: [0, 3, 4]

[ Current round 0 Train epoch: 0 ]
Round : 0 Party 0: Update Epoch: 0 [0/10000 (0%)]	Loss: 2.343992
Round : 0 Party 0: Update Epoch: 0 [1280/10000 (13%)]	Loss: 2.233247
Round : 0 Party 0: Update Epoch: 0 [2560/10000 (25%)]	Loss: 2.087997
Round : 0 Party 0: Update Epoch: 0 [3840/10000 (38%)]	Loss: 2.078186
Round : 0 Party 0: Update Epoch: 0 [5120/10000 (51%)]	Loss: 1.956667
Round : 0 Party 0: Update Epoch: 0 [6400/10000 (63%)]	Loss: 1.932101
Round : 0 Party 0: Update Epoch: 0 [7680/10000 (76%)]	Loss: 1.876982
Round : 0 Party 0: Update Epoch: 0 [8960/10000 (89%)]	Loss: 1.798764

Total adversarial train accuarcy: tensor(24.5400)

Total adversarial train loss: 2.005402970917617

[ Current round 0 Train epoch: 0 ]
Round : 0 Party 3: Update Epoch: 0 [0/10000 (0%)]	Loss: 2.379504
Round : 0 Party 3: Update Epoch: 0 [1280/10000 (13%)]	Loss: 2.205266
Round : 0 Party 3: Update Epoch: 0 [2560/10000 (25%)]	Loss: 2.181103
Round : 0 Party 3: Up

 50%|█████     | 1/2 [00:41<00:41, 41.57s/it]

The final model saved to:  ./save/Krum_0.031_cifar_customized_resnet18_2_1_nParties_3_iid__1000_0_model_2023-09-27T15-10-16.pt
Model Saved!
filepath  ./save/Krum_0.031_cifar_customized_resnet18_$2_1_nParties_3_iid_1000_0_2023-09-27T15-10-16.out
The output file saved to:  ./save/Krum_0.031_cifar_customized_resnet18_$2_1_nParties_3_iid_1000_0_2023-09-27T15-10-16.out
Attacker Indices: [3 2]
Non-Attacker Indices: [0, 1, 4]

[ Current round 1 Train epoch: 0 ]
Round : 1 Party 0: Update Epoch: 0 [0/10000 (0%)]	Loss: 1.889886
Round : 1 Party 0: Update Epoch: 0 [1280/10000 (13%)]	Loss: 1.713759
Round : 1 Party 0: Update Epoch: 0 [2560/10000 (25%)]	Loss: 1.654663
Round : 1 Party 0: Update Epoch: 0 [3840/10000 (38%)]	Loss: 1.725882
Round : 1 Party 0: Update Epoch: 0 [5120/10000 (51%)]	Loss: 1.694599
Round : 1 Party 0: Update Epoch: 0 [6400/10000 (63%)]	Loss: 1.599142
Round : 1 Party 0: Update Epoch: 0 [7680/10000 (76%)]	Loss: 1.655184
Round : 1 Party 0: Update Epoch: 0 [8960/10000 (89%)]	Loss: 1.

100%|██████████| 2/2 [01:23<00:00, 41.79s/it]

The output file saved to:  ./save/Krum_0.031_cifar_customized_resnet18_$2_1_nParties_3_iid_1000_0_2023-09-27T15-10-16.out


# Reproducibility

In [70]:
#Load model
# f_model_path='./save/fedavg_0.031_cifar_customized_resnet18_20_5_nParties_5_twoclassnoniid__1000_1.0_model_2023-07-26T22-02-39.pt'
net_glob_2 = build_model(args, img_size)
weights = torch.load(f_model_path)
net_glob_2.load_state_dict(weights) 
net_glob_2.to(args.device)
####
test_idxs = np.arange(len(dataset_test))
###
acc_test, adv_acc_test, (_) = test_img(net_glob_2, 
                                        dataset_test, 
                                        test_idxs, 
                                        args, 
                                        loss_fn)

This is customized_resnet18

Test set: 
Accuracy: 3821/10000 (38.21%)


FGSM -- Adversarial Test set as a classifier: 
Adv Accuracy: 51/10000 (0.51%)


Number of correct classified clean examples(as compared to clean predictions): 3821/10000

Number of correct classified adversarial examples(as compared to clean predictions): 51/10000


In [71]:
accuracy, Gaussian_acc, FGSM_acc_noise, exp_adv_noise = test_img_noise(net_glob_2, 
                                                            dataset_test,
                                                            test_idxs,
                                                            args, 
                                                            loss_fn)


Test set: 
Accuracy on benign test examples: 3821/10000 (38.21%)


Test set: 
Accuracy on Gaussin noise test examples: 5/10000 (0.05%)


Test accuracy for noisy FGSM: 
Adv Accuracy: 0.5099999904632568/10000 (0.51%)

